In [62]:
import pandas as pd
import numpy as np
import datetime
from openpyxl import load_workbook

In [63]:
SEASON = 12
MATCHDAY = 11

In [64]:
mc = pd.read_excel('data/match_'+str(MATCHDAY)+'.xlsx')
mc.drop('№', axis=1, inplace=True)
mc.rename(columns={'index': 'Name'}, inplace=True)

In [65]:
mc.columns

Index(['Name', 'Итог', 'рома – наполи', '0\n2', 'фулхэм – лидс', '1\n2',
       'ницца – марсель', '3\n0', 'селтик – рейнджерс', '1\n1',
       'аз алкмар – псв', '2\n0', 'вест хэм – арсенал', '3\n3',
       'бешикташ – фенербахче', '1\n1.1', 'фиорентина – милан', '2\n3',
       'брага – бенфика', '0\n2.1', 'лион – псж', '2\n4'],
      dtype='object')

In [66]:
#Заголовки нулевой таблицы для создания ДБ ставок
zeros_col = ['Total Points', 'Game Points', 'Bomb Points', 'Exact Score', 'Right Margin', 'Right Result', 'Wrong Result', 'Uniques']

matches = pd.read_excel('output/season_'+str(SEASON)+'.xlsx', sheet_name='matches', index_col=0)
players = pd.read_excel('output/season_'+str(SEASON)+'.xlsx', sheet_name='players', index_col=0)
season = pd.read_excel('output/season_'+str(SEASON)+'.xlsx', sheet_name='season', index_col=0)
teams = pd.read_excel('output/season_'+str(SEASON)+'.xlsx', sheet_name='teams', index_col=0)

In [67]:
def proceed_bet(player_id, match_id):
    """
    proceed_bet(player_id, match_id)
    Сверяет ставки с результатом матча.
    player_id = Id участника
    match_id = Id матча
    """
    player_name = players['Name'].iloc[player_id]
    
    h_team = matches['Hometeam'].loc[match_id]
    a_team = matches['Awayteam'].loc[match_id]
    match = str(h_team)+' - '+str(a_team)
    
    try:
        hscore_true = matches['H_score'].loc[match_id]
        ascore_true = matches['A_score'].loc[match_id]

        hscore_bet = int(bets['bet_hscore_'+str(match_id)].loc[player_id])
        ascore_bet = int(bets['bet_ascore_'+str(match_id)].loc[player_id])
        
    except ValueError:
        print(player_name, 'made no bet on', match)
        bets['points_'+str(match_id)].loc[player_id] = 0
        return
    
    true_score= str(hscore_true)+':'+str(ascore_true)
    bet_score= str(hscore_bet)+':'+str(ascore_bet)
    
    points_granted = 0
    
    if (hscore_true == hscore_bet and ascore_true == ascore_bet):
        points_granted = 5
        matches['Exact Score'].loc[match_id] += 1
        season['Exact Score'].loc[player_id] += 1
        bets['Exact Score'].loc[player_id] += 1
    elif (hscore_true - ascore_true) == (hscore_bet - ascore_bet):
        points_granted = 3
        matches['Right Margin'].loc[match_id] += 1
        season['Right Margin'].loc[player_id] += 1
        bets['Right Margin'].loc[player_id] += 1
    elif ((hscore_true > ascore_true) and (hscore_bet > ascore_bet)) or ((hscore_true < ascore_true) and (hscore_bet < ascore_bet)):
        points_granted = 2
        matches['Right Result'].loc[match_id] += 1
        season['Right Result'].loc[player_id] += 1
        bets['Right Result'].loc[player_id] += 1
    else:
        matches['Wrong Result'].loc[match_id] += 1
        season['Wrong Result'].loc[player_id] += 1
        bets['Wrong Result'].loc[player_id] += 1
    matches['Bets'].loc[match_id] += 1
    season['Bets'].loc[player_id] += 1
    bets['points_'+str(match_id)].loc[player_id] = points_granted
    return

def count_total(player_id):
    """
    Считает суммарное число баллов, суммируя значения с bets 
    """
    total = 0
    for i in range(11, bets.shape[1], 3):
        total += bets.loc[player_id].iloc[i]
    bets['Game Points'].loc[player_id] = total
    season['Game Points'].loc[player_id] += total
    print(bets.iloc[player_id, 0], total)
    
def find_uniques(match_ids):
    h_score = str(matches['H_score'].loc[match_ids])
    a_score = str(matches['A_score'].loc[match_ids])
    player_id = bets[bets['bet_hscore_'+str(match_ids)] == h_score][bets['bet_ascore_'+str(match_ids)] == a_score].index[0]
    bets['Uniques'].loc[player_id] += 1
    season['Uniques'].loc[player_id] += 1
    matches['Uniques'].iloc[match_ids] += 1
    return player_id

def get_match_ids(season=SEASON, matchday=MATCHDAY):
    ids = matches[matches['Season'] == season][matches['Matchday'] == matchday].index
    return_list = ['Name', 'Total Points', 'Game Points', 'Bomb Points', 'Exact Score', 'Right Margin', 'Right Result', 'Wrong Result', 'Uniques']
    for i in ids:
        return_list.append('bet_hscore_'+str(i))
        return_list.append('bet_ascore_'+str(i))
        return_list.append('points_'+str(i))
    return return_list

In [68]:
for match, score in zip(mc.columns[2::2], mc.columns[3::2]):
    hometeam, awayteam = match.split(' – ')[0], match.split(' – ')[1]
    h_score, a_score = score.split('\n')[0], score.split('\n')[1][0]
      
    if matches[matches['Matchday'] == MATCHDAY][matches['Hometeam'] == hometeam][matches['Awayteam'] == awayteam].shape[0] > 0:
        print(f'{hometeam} - {awayteam} уже есть')
    else:
        matches = matches.append({'Hometeam': hometeam, 'Awayteam': awayteam,
                                  'H_score': int(h_score), 'A_score': int(a_score),
                                  'Season': SEASON, 'Matchday': MATCHDAY, 'Date': datetime.datetime.now(), 'Bets': 0,
                                  'Exact Score': 0, 'Right Margin': 0, 'Right Result': 0, 
                                  'Wrong Result': 0, 'Uniques': 0}, ignore_index=True)
    
bets_columns = get_match_ids(SEASON,MATCHDAY)
bets = pd.DataFrame(pd.concat([mc['Name'], pd.DataFrame(np.zeros((mc['Name'].shape[0],8)), columns=zeros_col)], axis=1), columns=bets_columns)  

#Очень тупо, я повторяю цикл, потому что между этими операциями ме надо добавить БД для ставок
for match, score in zip(mc.columns[2::2], mc.columns[3::2]):
    hometeam, awayteam = match.split(' – ')[0], match.split(' – ')[1]
    h_score, a_score = score.split('\n')[0], score.split('\n')[1][0]    
    match_id = matches[matches['Hometeam'] == hometeam][matches['Awayteam'] == awayteam].index[0]   
    
    bets['bet_hscore_'+str(match_id)] = mc[match].str.split(':').str[0]
    bets['bet_ascore_'+str(match_id)] = mc[match].str.split(':').str[1]

<ipython-input-68-7dd6f4ee2aab>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if matches[matches['Matchday'] == MATCHDAY][matches['Hometeam'] == hometeam][matches['Awayteam'] == awayteam].shape[0] > 0:
<ipython-input-68-7dd6f4ee2aab>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if matches[matches['Matchday'] == MATCHDAY][matches['Hometeam'] == hometeam][matches['Awayteam'] == awayteam].shape[0] > 0:
<ipython-input-68-7dd6f4ee2aab>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if matches[matches['Matchday'] == MATCHDAY][matches['Hometeam'] == hometeam][matches['Awayteam'] == awayteam].shape[0] > 0:
<ipython-input-68-7dd6f4ee2aab>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if matches[matches['Matchday'] == MATCHDAY][matches['Hometeam'] == hometeam][matches['Awayteam'] == awayteam].shape[0] > 0:
<ipython-input-68-7dd6f4ee2aab>:5: UserWarning: Bool

In [69]:
for player in range(0,players['Name'].shape[0]):
    for match in matches[matches['Season']==SEASON][matches['Matchday']==MATCHDAY].index:
        proceed_bet(player,match)

H:\dev\py\python-3.8.5.amd64\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
H:\dev\py\python-3.8.5.amd64\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
H:\dev\py\python-3.8.5.amd64\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

Андреев Иван made no bet on фиорентина - милан
Белькин Сергей made no bet on фулхэм - лидс
Гришин Антон made no bet on фулхэм - лидс
Колодин Дмитрий made no bet on рома - наполи
Колодин Дмитрий made no bet on фулхэм - лидс
Колодин Дмитрий made no bet on ницца - марсель
Колодин Дмитрий made no bet on селтик - рейнджерс
Колодин Дмитрий made no bet on аз алкмар - псв
Колодин Дмитрий made no bet on вест хэм - арсенал
Колодин Дмитрий made no bet on бешикташ - фенербахче
Колодин Дмитрий made no bet on фиорентина - милан
Колодин Дмитрий made no bet on брага - бенфика
Колодин Дмитрий made no bet on лион - псж
Котов Александр made no bet on фулхэм - лидс
Котов Александр made no bet on ницца - марсель


H:\dev\py\python-3.8.5.amd64\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
H:\dev\py\python-3.8.5.amd64\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
H:\dev\py\python-3.8.5.amd64\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

In [70]:
mc

,Name,Итог,рома – наполи,0\n2,фулхэм – лидс,1\n2,ницца – марсель,3\n0,селтик – рейнджерс,1\n1,...,вест хэм – арсенал,3\n3,бешикташ – фенербахче,1\n1.1,фиорентина – милан,2\n3,брага – бенфика,0\n2.1,лион – псж,2\n4
0,Алферов Ян,10.0,1:2,2.0,2:1,0.0,1:3,0.0,0:1,0.0,...,1:1,3.0,3:1,0.0,1:2,3.0,2:2,0.0,1:1,0.0
1,Андреев Иван,11.0,1:2,2.0,2:1,0.0,2:2,0.0,1:1,5.0,...,1:3,0.0,2:1,0.0,NaN,0.0,1:1,0.0,1:4,2.0
2,Афанасьев Владимир,10.0,1:1,0.0,2:1,0.0,1:2,0.0,2:1,0.0,...,1:1,3.0,1:2,0.0,1:2,3.0,1:2,2.0,1:2,2.0
3,Белькин Сергей,9.0,1:2,2.0,NaN,0.0,0:0,0.0,1:0,0.0,...,3:1,0.0,0:0,3.0,0:3,2.0,1:0,0.0,1:1,0.0
4,Гришин Антон,12.0,1:1,0.0,NaN,0.0,1:1,0.0,1:1,5.0,...,2:1,0.0,1:1,5.0,1:1,0.0,1:1,0.0,1:2,2.0
5,Данилов Евгений,18.0,0:2,5.0,2:3,3.0,1:2,0.0,0:2,0.0,...,1:2,0.0,3:1,0.0,1:2,3.0,0:2,5.0,0:3,2.0
6,Еременко Владислав,21.0,1:2,2.0,1:2,5.0,1:1,0.0,0:2,0.0,...,1:2,0.0,1:1,5.0,0:2,2.0,1:2,2.0,1:3,3.0
7,Жигалов Александр,21.0,0:2,5.0,0:1,3.0,0:2,0.0,1:0,0.0,...,0:1,0.0,1:0,0.0,1:0,0.0,0:2,5.0,0:1,2.0
8,Зубатов Михаил,14.0,1:2,2.0,1:2,5.0,1:2,0.0,1:2,0.0,...,1:2,0.0,1:2,0.0,1:3,2.0,1:2,2.0,1:3,3.0
9,Колодин Дмитрий,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
bets['Name']

0             Алферов Ян
1           Андреев Иван
2     Афанасьев Владимир
3         Белькин Сергей
4           Гришин Антон
5        Данилов Евгений
6     Еременко Владислав
7      Жигалов Александр
8         Зубатов Михаил
9        Колодин Дмитрий
10        Конаков Никита
11       Котов Александр
12    Криеванс Владислав
13      Кучеренко Никита
14      Лесик Константин
15          Малевич Егор
16       Махмудов Руслан
17           Минеев Иван
18         Муратов Игорь
19        Невский Леонид
20     Новиков Александр
21        Оксанич Кирилл
22     Сибиряков Георгий
23       Титаренко Антон
24      Фёдоров Владимир
25     Чапаров Александр
26          Шевчук Антон
Name: Name, dtype: object

In [74]:
def find_match_id(tour):
    return 11+(tour-1)*3

In [78]:
bets.iloc[17,find_match_id(3)] = 3


In [79]:
for i in matches[matches['Season']==SEASON][matches['Matchday']==MATCHDAY][matches['Exact Score']==1].index:
    player_id = find_uniques(i)
    bets['points_'+str(i)].loc[player_id] += 1
    
for player in range(0,players['Name'].shape[0]):
    count_total(player)

Алферов Ян 10.0
Андреев Иван 11.0
Афанасьев Владимир 10.0
Белькин Сергей 9.0
Гришин Антон 12.0
Данилов Евгений 18.0
Еременко Владислав 21.0
Жигалов Александр 21.0
Зубатов Михаил 14.0
Колодин Дмитрий 0.0
Конаков Никита 9.0
Котов Александр 8.0
Криеванс Владислав 12.0
Кучеренко Никита 14.0
Лесик Константин 13.0
Малевич Егор 15.0
Махмудов Руслан 8.0
Минеев Иван 11.0
Муратов Игорь 7.0
Невский Леонид 17.0
Новиков Александр 7.0
Оксанич Кирилл 18.0
Сибиряков Георгий 10.0
Титаренко Антон 13.0
Фёдоров Владимир 10.0
Чапаров Александр 13.0
Шевчук Антон 8.0


<ipython-input-79-c524e9ee286a>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  for i in matches[matches['Season']==SEASON][matches['Matchday']==MATCHDAY][matches['Exact Score']==1].index:
<ipython-input-67-0ecd95017322>:69: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  player_id = bets[bets['bet_hscore_'+str(match_ids)] == h_score][bets['bet_ascore_'+str(match_ids)] == a_score].index[0]
H:\dev\py\python-3.8.5.amd64\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


### Bombers

In [80]:
bombs = pd.read_excel('output/bombs_bets.xlsx', index_col=0, sheet_name=str(MATCHDAY))


In [81]:
# Сгрупировал таблицу в формате имя - очки и присвоил в новую переменную
bombs_points = bombs[bombs['Points']>0].groupby(['Name']).sum().sort_values(by=['Points'], ascending=False)

# Теперь итерируем по всей таблице bets. Если имя есть в bomb_points обновляем значение таблицы Bomb Points. 
# Важно вызывать через loc чтобы не вызывать копию, а работать напрямую с табилцей
print('Очки полученные за бомбардиров')
for row in np.arange(bets.shape[0]):
    try:
        bets.loc[row,'Bomb Points'] = bombs_points['Points'][bets.loc[row,'Name']]
        print(bets.loc[row,'Bomb Points'], bets.loc[row,'Name'])
    except KeyError:
        pass

Очки полученные за бомбардиров
2.0 Алферов Ян
4.0 Андреев Иван
6.0 Афанасьев Владимир
6.0 Гришин Антон
6.0 Данилов Евгений
8.0 Еременко Владислав
2.0 Жигалов Александр
8.0 Зубатов Михаил
2.0 Криеванс Владислав
6.0 Кучеренко Никита
4.0 Лесик Константин
6.0 Малевич Егор
4.0 Минеев Иван
6.0 Муратов Игорь
10.0 Невский Леонид
2.0 Новиков Александр
4.0 Оксанич Кирилл
6.0 Сибиряков Георгий
2.0 Титаренко Антон
6.0 Фёдоров Владимир
8.0 Чапаров Александр
6.0 Шевчук Антон


In [82]:
bets['Total Points'] = bets['Game Points'] + bets['Bomb Points']

season['Bomb Points'] += bets['Bomb Points']
season['Total Points'] = season['Game Points'] + season['Bomb Points']

In [83]:
bets.sort_values(by=['Total Points'], ascending=False)[['Name','Total Points']]

,Name,Total Points
6,Еременко Владислав,29.0
19,Невский Леонид,27.0
5,Данилов Евгений,24.0
7,Жигалов Александр,23.0
21,Оксанич Кирилл,22.0
8,Зубатов Михаил,22.0
25,Чапаров Александр,21.0
15,Малевич Егор,21.0
13,Кучеренко Никита,20.0
4,Гришин Антон,18.0


In [84]:
season.sort_values(by=['Total Points'], ascending=False)[['Name','Total Points']]

,Name,Total Points
21,Оксанич Кирилл,215.0
14,Лесик Константин,212.0
19,Невский Леонид,205.0
7,Жигалов Александр,204.0
15,Малевич Егор,200.0
4,Гришин Антон,199.0
22,Сибиряков Георгий,186.0
12,Криеванс Владислав,183.0
26,Шевчук Антон,175.0
13,Кучеренко Никита,174.0


In [85]:
for match_id in matches[matches['Season']==SEASON][matches['Matchday']==MATCHDAY].index:
    match = matches.iloc[match_id,:]
    hometeam, awayteam = match[0], match[1]
    m_s = matches.iloc[match_id,:]    #Match_series
    for team in [hometeam, awayteam]:
        # Какое говно)))))
        if team == m_s['Hometeam']:
            if m_s['H_score'] > m_s['A_score']:
                win, draw, loss = 1,0,0
            elif m_s['H_score'] < m_s['A_score']:
                win, draw, loss = 0,0,1
            else:
                win, draw, loss = 0,1,0
        elif team == m_s['Awayteam']:
            if m_s['H_score'] > m_s['A_score']:
                win, draw, loss = 0,0,1
            elif m_s['H_score'] < m_s['A_score']:
                win, draw, loss = 1,0,0
            else:
                win, draw, loss = 0,1,0
                      
        if team in teams['Team'].values:
            team_index = teams[teams['Team'] == team].index[0]
            teams.iloc[team_index,2] += 1
            teams.iloc[team_index,3] += win
            teams.iloc[team_index,4] += draw
            teams.iloc[team_index,5] += loss
            teams.iloc[team_index,6] += m_s['Bets']
            teams.iloc[team_index,7] += m_s['Exact Score']
            teams.iloc[team_index,8] += m_s['Right Margin']
            teams.iloc[team_index,9] += m_s['Right Result']
            teams.iloc[team_index,10] += m_s['Wrong Result']
            teams.iloc[team_index,11] += m_s['Uniques']
        else:
            teams = teams.append({'Team': team, 'Matches': 1, 'Wins': win, 'Draws': draw, 'Losses': loss,
                                 'Bets': m_s['Bets'], 'Exact Score': m_s['Exact Score'], 'Right Margin': m_s['Right Margin'], 
                                 'Right Result': m_s['Right Result'], 'Wrong Result': m_s['Wrong Result'], 
                                 'Uniques': m_s['Uniques']}, ignore_index=True)

In [86]:
teams

,Team,Country,Matches,Wins,Draws,Losses,Bets,Exact Score,Right Margin,Right Result,Wrong Result,Uniques
0,леванте,NaN,1,0,1,0,25,1,5,0,19,1
1,гранада,NaN,3,1,1,1,77,8,5,17,47,2
2,майнц,NaN,1,1,0,0,27,1,2,1,23,1
3,унион берлин,NaN,3,1,1,1,76,10,5,1,60,1
4,ньюкасл юнайтед,NaN,1,1,0,0,27,0,4,3,20,0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,ницца,NaN,1,1,0,0,25,0,0,1,24,0
96,селтик,NaN,1,0,1,0,26,8,1,0,17,0
97,аз алкмар,NaN,1,1,0,0,26,1,2,7,16,1
98,бешикташ,NaN,1,0,1,0,26,3,2,0,21,0


In [87]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
book = load_workbook('output/bets.xlsx')
writer = pd.ExcelWriter('output/bets.xlsx', engine='openpyxl')
writer.book = book

# Write each dataframe to a different worksheet.
bets.to_excel(writer, sheet_name=str(MATCHDAY))

writer.save()
writer.close()

In [88]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(f'output/season_{str(SEASON)}.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
season.to_excel(writer, sheet_name='season')
matches.to_excel(writer, sheet_name='matches')
players.to_excel(writer, sheet_name='players')
teams.to_excel(writer, sheet_name='teams')
#scorers.to_excel(writer, sheet_name='scorers_18')

writer.save()